In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
%matplotlib inline

import os
import sys
import datetime

import pyvips as pv
import wandb
import cv2

import warnings; warnings.filterwarnings('ignore')

sys.path.append('..')
from config import CFG
CFG = CFG()
pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.max_columns', CFG.NCOLS)
pd.set_option('display.max_rows', CFG.NROWS)

In [2]:
train_meta = pd.read_csv(f'{CFG.DATA_PATH}/train_metadata.csv')
test_meta = pd.read_csv(f'{CFG.DATA_PATH}/test_metadata.csv')
train_labels = pd.read_csv(f'{CFG.DATA_PATH}/train_labels.csv')
test_labels = pd.read_csv(f'{CFG.DATA_PATH}/test_labels.csv')
annot = pd.read_csv(f'{CFG.DATA_PATH}/annotations.csv')

In [4]:
annot.head(3)

,filename,Annotation_ID,Annotation_Term,Area,Perimeter,WKT_geometry,opencv_geometry,us_tif_url,eu_tif_url,as_tif_url
0,9k10x5h3.tif,519366,Breslow,0.00,1.71,"LINESTRING (19408 90416, 19632 90416, 19760 90...","LINESTRING (19408 33488, 19632 33488, 19760 33...",s3://drivendata-competition-visiomel-public-us...,s3://drivendata-competition-visiomel-public-eu...,s3://drivendata-competition-visiomel-public-as...
1,9k10x5h3.tif,519333,Normal sub-cutaneous tissue,0.00,49.29,"LINESTRING (53696 20672, 54336 20672, 54464 20...","LINESTRING (53696 103232, 54336 103232, 54464 ...",s3://drivendata-competition-visiomel-public-us...,s3://drivendata-competition-visiomel-public-eu...,s3://drivendata-competition-visiomel-public-as...
2,9k10x5h3.tif,519298,Normal dermis,0.00,26.66,"LINESTRING (30400 111552, 30656 111552, 31552 ...","LINESTRING (30400 12352, 30656 12352, 31552 12...",s3://drivendata-competition-visiomel-public-us...,s3://drivendata-competition-visiomel-public-eu...,s3://drivendata-competition-visiomel-public-as...


In [71]:
!s5cmd --no-sign-request ls s3://drivendata-competition-visiomel-public-us/images/ > list.txt
with open('list.txt') as f:
    l = f.readlines()
    l = l[1:]
l = pd.Series(l).apply(lambda x: x.split(' ')[-1].split('.')[0])
dl = os.listdir(f'{CFG.DATA_PATH}/images')
dl = pd.Series(dl).apply(lambda x: x.split('.')[0])
dlist = l[~l.isin(dl)]
dlist = dlist.apply(lambda x: x + '.tif')
train_meta = train_meta[train_meta['filename'].isin(dlist.values.tolist())]

In [3]:
def dl_img(row):
    path = row[10]
    file = row[1]
    new = row[1].split('.')[0]
    !s5cmd --no-sign-request cp {path} /mnt/disk1/visiomel-melanoma-runtime/data/images/

    img = pv.Image.new_from_file(f'{CFG.DATA_PATH}/images/{file}', page=4)
    img.write_to_file(f'{CFG.DATA_PATH}/images/{new}.jpg')
    del img
    os.remove(f'{CFG.DATA_PATH}/images/{file}')
    gc.collect()

In [3]:
def dl_img(dlpath):
    fname = dlpath.split('/')[-1]
    !s5cmd --no-sign-request cp {dlpath} /mnt/disk1/visiomel-melanoma-runtime/data/originals/
    !s5cmd --endpoint-url https://storage.googleapis.com cp /mnt/disk1/visiomel-melanoma-runtime/data/originals/{fname} s3://visiomel/originals/

In [4]:
import gc
for row in train_meta.itertuples():
    dl_img(row[10])

cp s3://drivendata-competition-visiomel-public-us/images/1u4lhlqb.tif /mnt/disk1/visiomel-melanoma-runtime/data/originals/1u4lhlqb.tif
cp /mnt/disk1/visiomel-melanoma-runtime/data/originals/1u4lhlqb.tif s3://visiomel/originals/1u4lhlqb.tif
cp s3://drivendata-competition-visiomel-public-us/images/rqumqnfp.tif /mnt/disk1/visiomel-melanoma-runtime/data/originals/rqumqnfp.tif
cp /mnt/disk1/visiomel-melanoma-runtime/data/originals/rqumqnfp.tif s3://visiomel/originals/rqumqnfp.tif
cp s3://drivendata-competition-visiomel-public-us/images/bu5xt1xm.tif /mnt/disk1/visiomel-melanoma-runtime/data/originals/bu5xt1xm.tif
cp /mnt/disk1/visiomel-melanoma-runtime/data/originals/bu5xt1xm.tif s3://visiomel/originals/bu5xt1xm.tif
cp s3://drivendata-competition-visiomel-public-us/images/dibvu7wk.tif /mnt/disk1/visiomel-melanoma-runtime/data/originals/dibvu7wk.tif
cp /mnt/disk1/visiomel-melanoma-runtime/data/originals/dibvu7wk.tif s3://visiomel/originals/dibvu7wk.tif
cp s3://drivendata-competition-visiomel-